In [ ]:
import os
# import shutil
import warnings
import sys
# import glob
import re
from datetime import date, datetime, timedelta
# from dateutil.relativedelta import relativedelta
from collections import defaultdict
from collections import OrderedDict
from scipy import stats
# import scipy.io
import csv
import xlsxwriter
from statistics import mode
import math
import time
from mea_functions import averageMetricsAndDates
from mea_functions import data_and_date_reconstruction
from mea_functions import averageAmps
# from mea_functions import statistics_chart
from tqdm.notebook import tqdm_notebook
import numpy as np
import pandas as pd
# import seaborn as sns
import openpyxl as xl
from copy import copy
# from copy import deepcopy
%matplotlib widget
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
# import statsmodels.api as sm
# from bioinfokit.analys import stat
# from statsmodels.formula.api import ols
# import pingouin as pg
# from statsmodels.graphics.factorplots import interaction_plot
# from brokenaxes import brokenaxes
# import matplotlib.patheffects as pe
from matplotlib import container
# import concurrent.futures
# from itertools import starmap
# import multiprocessing as mp
# from brokenaxes import brokenaxes
from pptx import Presentation
from pptx.util import Inches, Pt
import tkinter as tk
import tkinter.filedialog as fd

In [ ]:
# hide these warnings
warnings.filterwarnings(action='ignore', message='.*converting a masked element to nan.*')

# Some plotting presets
# plt.rc("errorbar", capsize=3)
plt.rc("figure", dpi=70)
plt.rc("savefig", dpi=40, facecolor="white", bbox="tight")

In [ ]:
#print python version
python_version = sys.version
print(python_version)

## Choose analysis files
initial_dir = "C:\\"
print("Please select the files to analyze:")

time.sleep(0.25)
root = tk.Tk()
root.filenames = fd.askopenfilenames(initialdir=initial_dir,
                                     title="Please select the files to analyze",
                                     filetypes=(("csv files", "*.csv"), ("all files", "*.*")))
input_files = [os.path.basename(f) for f in root.filenames]
input_dir = os.path.dirname(root.filenames[0])
root.destroy()

print("Please choose the output folder:")
time.sleep(0.25)

root = tk.Tk()
root.filenames = fd.askdirectory(initialdir=input_dir,
                                 title="Please select the output folder")
output_dir = root.filenames
root.destroy()

os.chdir(input_dir)
print('Working directory is:', input_dir, '\n')
print('Output folder is:', output_dir,'\n')

In [ ]:
# Couple each file with its date of creation
messydatesdict = OrderedDict()
for fl in input_files:
    if '_spike_' not in fl:
        with open(fl, encoding='UTF-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            row = list(csv_reader)
            for lin in row[75:76]:
                if any('Analysis Duration (s):' in e.strip() for e in lin):
                    duration = round(float(lin[1])/2) # Use the middle of the recording as its new timestamp
            for lin in row[5:7]:
                if any('Original File Time' in e.strip() for e in lin):
                    stamp = lin[1].split(' ')[0]
                    clock = lin[1].split(' ')[1]
                    year = int(stamp.split('/')[2])
                    month = int(stamp.split('/')[0])
                    day = int(stamp.split('/')[1])
                    hour = int(clock.split(':')[0])
                    minute = int(clock.split(':')[1])
                    second = int(clock.split(':')[2])
                    messydatesdict[fl] = datetime(year,month,day,hour,minute,second) + timedelta(seconds=duration)

# Sort file names by their date of creation
messydatesdict = dict(sorted(messydatesdict.items(), key=lambda x: datetime.strptime(str(x[1]), '%Y-%m-%d %H:%M:%S')))
for k, v in messydatesdict.items():
    print(k, v)

In [ ]:
# "clean" file names
cleandatesdict = OrderedDict()
datesforaveragemerging = OrderedDict()
for key, value in messydatesdict.items():
    new_key = key.split('.')[0].replace('(000)(000)', '').replace('(000)', '').replace('(00', ' ').replace('(0', ' ').replace(')', '')
    cleandatesdict[new_key] = str(value)
    datesforaveragemerging[new_key] = value
baselinedatesdict = {}
pretreatmentdatesdict = {}
acutedatesdict = {}

#Create baseline, pretreatment and acute dictionaries and find their indexes within the experiments' time points
indexes_bl = []
indexes_pre = []
indexes_ac = []
for k, v in cleandatesdict.items():
    if 'bl' in k:
        baselinedatesdict[k] = v
        indexes_bl.append(list(cleandatesdict.keys()).index(k))
    elif 'pretreatment' in k.lower():
        pretreatmentdatesdict[k] = v
        indexes_pre.append(list(cleandatesdict.keys()).index(k))
    elif 'ac' in k:
        acutedatesdict[k] = v
        indexes_ac.append(list(cleandatesdict.keys()).index(k))
    print(k, v)
# Dump the timepoints that need to be merged and the ones that not to different dictionaries
datestobemergeddict = OrderedDict(baselinedatesdict|pretreatmentdatesdict|acutedatesdict)
datesnottobemergeddict = {}
for k, v in cleandatesdict.items():
    if k not in datestobemergeddict.keys():
        datesnottobemergeddict[k] = v

In [ ]:
# append desired parameters' data to dataList from the advanced metrics files
rowsChulas = np.r_[67,69,71,72,168:186 + 1] - 1
dataList = []
normal = []
duration_in_sec = False
durations = []
for fl in messydatesdict.keys():
    with open(fl, encoding='UTF-8') as csv_file:
        temporary = csv.reader(csv_file, delimiter=',')
        for i, row_content in enumerate(temporary):
            if 'Analysis Duration (s): ' in row_content:
                durations.append(float(row_content[1]))
                
for fl in messydatesdict.keys():
    corresponding_index = 'indexes_'+next(subs for subs in ['bl','ac','pre'] if subs in fl) \
                        if any(_ in fl for _ in ['bl','ac','pre']) else 'normal'
    with open(fl, encoding='UTF-8') as csv_file:
        singleCSV = []
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row_content in enumerate(csv_reader):
            if 'Analysis Duration (s): ' in row_content:
                duration_in_sec = float(row_content[1])
            if i in rowsChulas:                
                if len(globals()[corresponding_index]) <= 1: # If the currently open fl is not bl, pre or ac or it is but there is only one of its type in the experiment, do:       
                    if row_content[0] == 'Treatment':
                        row_content[0] = 'Treatment - Well'
                        singleCSV.append(row_content)
                    elif (row_content[0] == 'Number of Spikes' or row_content[0] == 'Number of Bursts' or
                    row_content[0] == 'Number of Network Bursts' or row_content[0] == 'Inter-Burst Interval - Avg (s)' or
                    row_content[0] == 'Inter-Burst Interval - Std (s)'):
                        if duration_in_sec >= mode(durations):
                            singleCSV.append([round(float(x) / (duration_in_sec/mode(durations))) if x != '' else '' for x in row_content[1:]]) # Divide all values in row[1:] by mode(time span) of the recordings
                            singleCSV[-1].insert(0, row_content[0])
                        else:
                            singleCSV.append(row_content)
                    elif row_content[0] == 'Measurement':
                        row_content[0] = 'Electrode'
                        singleCSV.append(row_content)
                    else:
                        singleCSV.append(row_content)                    
                    
                elif len(globals()[corresponding_index]) > 1: # Elif the currently open fl is bl, pre or ac and there are more than one of its type in the experiment, do:
                    if row_content[0] == 'Treatment':
                        row_content[0] = 'Treatment - Well'
                        singleCSV.append(row_content)
                    elif row_content[0] == 'Measurement':
                        row_content[0] = 'Electrode'
                        singleCSV.append(row_content)
                    else:
                        singleCSV.append(row_content)
        dataList.append(singleCSV)

for i, row_content in enumerate(dataList):
    dataList[i][2][1:] = [c + ' ' + t if c != '' else t for t, c in zip(dataList[i][2][1:], dataList[i][3][1:])]
    dataList[i].insert(3, ['Treatment - Electrode'] + [dataList[i][2][a+1] for a, w in enumerate(dataList[i][0][1:]) for e in dataList[i][4][1:] if e[:2] == w])
    dataList[i].insert(2, ['Electrode Coloring'] + [dataList[i][1][a+1] for a, w in enumerate(dataList[i][0][1:]) for e in dataList[i][5][1:] if e[:2] == w])
    
'''
In the future, concentration values will be included in the name of the treatment
'''

In [ ]:
# Call a function to calculate metric averages per hour and their new respective timestamps
dataListWholeAvgbl, dataListperHourbl, defdatesbl = averageMetricsAndDates(indexes_bl,dataList,cleandatesdict,datesforaveragemerging,'bl')
dataListWholeAvgpre, dataListperHourpre, defdatespre = averageMetricsAndDates(indexes_pre,dataList,cleandatesdict,datesforaveragemerging,'pretreatment')
dataListWholeAvgac, dataListperHourac, defdatesac = averageMetricsAndDates(indexes_ac,dataList,cleandatesdict,datesforaveragemerging,'ac')

In [ ]:
# Call a function to reconstruct 'dataList' and 'cleandatesdict' IF NECESSARY
dataList, cleandatesdict = data_and_date_reconstruction(dataList, messydatesdict, cleandatesdict,
                                                        datesnottobemergeddict,
                                                        dataListWholeAvgbl, dataListperHourbl, defdatesbl,
                                                        dataListWholeAvgpre, dataListperHourpre, defdatespre,
                                                        dataListWholeAvgac, dataListperHourac, defdatesac)

In [ ]:
# manually define a dictionary of shortened parameter names for the excel sheet names

excel_friendly = {
                'Well':'Well',
                'Well Coloring':'Well Coloring',
                'Electrode Coloring':'Electrode Coloring',
                'Treatment - Well':'Treatment - Well',
                'Treatment - Electrode':'Treatment - Electrode',
                'Concentration':'Concentration',
                'Electrode':'Electrode',
                'Number of Spikes':'Number_of_Spikes',
                'Mean Firing Rate (Hz)':'Mean_Firing_Rate_Hz',
                'ISI Coefficient of Variation':'ISI_Coefficient_of_Variation',
                'Number of Bursts':'Number_of_Bursts',
                'Burst Duration - Avg (s)':'Burst_Duration_Avg_s',
                'Burst Duration - Std (s)':'Burst_Duration_Std_s',
                'Number of Spikes per Burst - Avg':'Spikes_per_Burst_Avg',
                'Number of Spikes per Burst - Std':'Spikes_per_Burst_Std',
                'Mean ISI within Burst - Avg':'Mean_ISI_within_Burst_Avg',
                'Mean ISI within Burst - Std':'Mean_ISI_within_Burst_Std',
                'Median ISI within Burst - Avg':'Median_ISI_within_Burst_Avg',
                'Median ISI within Burst - Std':'Median_ISI_within_Burst_Std',
                'Inter-Burst Interval - Avg (s)':'Inter_Burst_Interval_Avg_s',
                'Inter-Burst Interval - Std (s)':'Inter_Burst_Interval_Std_s',
                'Burst Frequency (Hz)':'Burst_Frequency_Hz',
                'IBI Coefficient of Variation':'IBI_Coefficient_of_Variation',
                'Normalized Duration IQR':'Normalized Duration IQR',
                'Burst Percentage':'Burst_Percentage',
                }

# Check all names are <=31 chars long
for i in excel_friendly.values():
    if len(i) > 31:
        print(i+ ' is too long of an excel worksheet name')
print('Number of electrode advanced metrics =', len(excel_friendly))

In [ ]:
# Assign each day's metrics in dataList to their corresponding day in cleandatesdict and dump the info into the "data" dictionary
data = defaultdict(dict)

for day, ds in zip(cleandatesdict.keys(), dataList):
    for row in ds:
        key = excel_friendly[row[0]]
        data[day][key] = row[1:]

if any("bl" in k.lower() for k in data.keys()) or any("pretreatment" in k.lower() for k in data.keys()):
    # If there is a baseline or pretreatment, idx is the sorted order of the electrode treatments in the last timepoint
    idx = np.argsort(data[day]['Treatment - Electrode'])#merging treatment and concentration values within Treatment key
else:
    # Else, idx is the sorted order of the electrode coloring of the last timepoint (as coloring must follow some criteria)
    idx = np.argsort(data[day]['Electrode Coloring'])   
print('\nIndex of sorted electrode treatments is: ', idx)

In [ ]:
# Extract day of treatment from the name of the first 'ac' time point that received a treatment
day_of_treatment = False

if any("ac" in k.lower() for k in data.keys()):
    acute = [k for k in list(cleandatesdict.keys()) if 'ac' in k[:3]][0]
    day_of_treatment = acute.split(' ')[1].replace('div','')
    print('Treatment was administered in time point:', acute)
print('Day of treatment is: DIV',day_of_treatment)

# Calculate relative time from each time point to the acute treatment (time 0) if present
def_labels = {}
realistic_x_axis = {}
if any("ac" in k.lower() for k in data.keys()):
    act = datetime.strptime(cleandatesdict[acute], '%Y-%m-%d %H:%M:%S')
else:
    act = datetime.strptime(list(cleandatesdict.values())[0], '%Y-%m-%d %H:%M:%S')
for key in cleandatesdict.keys():
    now = datetime.strptime(cleandatesdict[key], '%Y-%m-%d %H:%M:%S')
    diff = now - act
    hours = diff.days*24 + diff.seconds/3600
    minutes = abs(hours*60) % 60 # abs not working the same for pos and neg values, that is why abs(hours). No negative sign in the pre-acute time points for this reason
    seconds = abs(hours*3600) % 60
    def_labels[key] = ["{}: {}".format(key, "%d%s%02d%s%02d%s" % (hours,'h ',minutes,'min ',seconds,'sec'))]
    realistic_x_axis[key] = hours

print('\ndef_labels:\n')
for k, v in def_labels.items():
    print(k+':\t', ",".join(v))

print('\nrealistic_x_axis:\n')
for k, v in realistic_x_axis.items():
    print(k+':\t', v, 'hours')

In [ ]:
# Create 'norms' dict and 'data' dict. Norms contains either the 'Whole averaged bl' metrics or the 'Whole averaged pre' metrics, depending on the experiment's structure
if any("pretreatment" in k.lower() for k in cleandatesdict.keys()):
    baseline = [key for key in cleandatesdict.keys() if 'pretreatment' in key.lower()][0]
    datafornorms = defaultdict(dict)
    for row in dataListWholeAvgpre:
        key = excel_friendly[row[0]]
        datafornorms[baseline][key] = row[1:]
    norms = dict()
    for parameter, quantity in datafornorms[baseline].items(): 
        if parameter not in ['Well', 'Well Coloring', 'Electrode Coloring', 'Treatment - Well', 'Treatment - Electrode', 'Concentration', 'Electrode']:
            norms[parameter] = []
            for val in quantity:
                try:
                    norms[parameter].append(float(val))
                except ValueError as e:
                    # print(f'{e} in {parameter} of {baseline}, appending None to norms')
                    norms[parameter].append(None)

elif any("bl" in k.lower() for k in cleandatesdict.keys()) and not any('pretreatment' in k.lower() for k in cleandatesdict.keys()):
    baseline = [key for key in cleandatesdict.keys() if 'bl' in key.lower()][0]
    datafornorms = defaultdict(dict)
    for row in dataListWholeAvgbl:
        key = excel_friendly[row[0]]
        datafornorms[baseline][key] = row[1:]
    norms = dict()
    for parameter, quantity in data[baseline].items(): 
        if parameter not in ['Well', 'Well Coloring', 'Electrode Coloring', 'Treatment - Well', 'Treatment - Electrode', 'Concentration', 'Electrode']:
            norms[parameter] = []
            for val in quantity:
                try:
                    norms[parameter].append(float(val))
                except ValueError as e:
                    # print(f'{e} in {parameter} of {baseline}, appending None to norms')
                    norms[parameter].append(None)
print(f'Baseline time point is {baseline}')

In [ ]:
# create excel workbook
excelName = os.path.join(output_dir, 'Tables_elec_multi.xlsx')
with xlsxwriter.Workbook(excelName, options= {'nan_inf_to_errors': True}) as workbook:
    with tqdm_notebook(total=len(data.values()), desc="Writing advanced metrics workbook rows on " + str(time.ctime(time.time()))) as pbar:
        styles = dict()
        params = data[list(cleandatesdict.keys())[0]]
        sheets_names = []
        for key in params.keys():
            if key not in ['Well', 'Well Coloring', 'Electrode Coloring', 'Treatment - Well', 'Treatment - Electrode', 'Concentration', 'Electrode']:
                sheets_names.append(key)
        for name in sheets_names:
            workbook.add_worksheet(name)
            workbook.add_worksheet(name='n{}'.format(name))
        sheets = [sh for sh in workbook.worksheets() if sh.name in sheets_names]
        norm_sheets = [sh for sh in workbook.worksheets() if sh.name not in sheets_names]
        for i, day_data in enumerate(data.values()):
            pbar.update(1)
            for color, treat in zip(day_data['Electrode Coloring'], day_data['Treatment - Electrode']):
                if not treat in styles:
                    styles[treat] = workbook.add_format({'bg_color': color})
                # styles[''] = workbook.add_format({'bg_color': '#00FF00'}) # define not-treated condition's color
            for sh in sheets:
                sh.write(i + 2, 0, list(cleandatesdict.keys())[i])
                sh.write(1, 0, 'Treatment')
                sh.write(0, 0, 'Electrode')
                for j in range(len(day_data['Electrode'])):
                    sh.write(0, j + 1, day_data['Electrode'][idx[j]])
                    treatment = day_data['Treatment - Electrode'][idx[j]]
                    sh.write(1, j + 1, treatment)
                    raw = day_data[sh.name][idx[j]]
                    try:
                        val = float(raw)
                        if val == 0:
                            val = 0
                    except ValueError as e:
                        # print(str(e).replace(':', ' in') + " {}, {}, {}".format(list(cleandatesdict.keys())[i],
                        #                                     sh.name,
                        #                                     day_data['Well'][idx[j]]))
                        val = 0
                    sh.write(i + 2, j + 1, val, styles[day_data['Treatment - Electrode'][idx[j]]])
            
            # write normalized values
            for sh in norm_sheets:
                sh.write(i + 2, 0, list(cleandatesdict.keys())[i])
                sh.write(1, 0, 'Treatment')
                sh.write(0, 0, 'Electrode')
                for j in range(len(day_data['Electrode'])):
                    sh.write(0, j + 1, day_data['Electrode'][idx[j]])
                    treatment = day_data['Treatment - Electrode'][idx[j]]
                    sh.write(1, j + 1, treatment)
                    raw = day_data[sh.name[1:]][idx[j]]
                    try:
                        if "pretreatment" in baseline.lower() and 'bl' not in list(cleandatesdict.keys())[i]:
                            val = float(raw) / norms[sh.name[1:]][idx[j]]
                        elif "pretreatment" in baseline.lower() and 'bl' in list(cleandatesdict.keys())[i]:
                            val = float(raw)
                        else: # if no pretreatment present
                            val = float(raw) / norms[sh.name[1:]][idx[j]]                            
                        if val == 0 or val > 4:
                            val = 0
                    except ValueError:
                        val = 0
                    except TypeError:
                        val = 0
                    except ZeroDivisionError: # if a well has active electrodes
#                          in the baseline, a ZeroDivisioError will prompt when
#                          trying to normalize (will try to divide float by 0)
#                          print("{} {} {} {}".format('Cannot divide',str(raw),'by',str(norms[sh.name[1:]][idx[j]])))
                        val = 0
                    sh.write(i + 2, j + 1, val, styles[day_data['Treatment - Electrode'][idx[j]]])
print(f'Created {excelName}')
"""A FileNotFound error prompting here means the name is too long for an excel workbook and it could not be initialized"""

In [ ]:
'''Now working on the spike amplitude files...'''

# Sort full spike_files by date of creation
filenames = []
for n, d in messydatesdict.items():
    filenames.append(f"{n.split('.csv')[0]}{'_spike_list.csv'}")
    print(filenames[-1])

In [ ]:
# create data structures
nombresListAmps = []
ampdata = dict()
electrode_info = dict()
electrodes = dataList[0][6][1:]

# add ampdata to structures
for fname in tqdm_notebook(filenames, desc = "Coupling amplitude values with their day, electrode, treatment and treatment concentration on " + str(time.ctime(time.time()))):
    # append short file names to "nombresListAmps"
    if 'on' in fname and not bool(re.search('\(00[1-9]\)|\([0-9][1-9][0-9]\)|\([1-9][0-9][0-9]\)', fname)):
        day = fname.split('.')[0].replace('(000)(000)', '').replace('(000)', '').replace('_spike_list', '')
    else:    
        day = fname.split('.')[0].replace('(000)', '').replace('_spike_list', '').replace('(00', ' ').replace('(0', ' ').replace(')', '')
    print(day)
    nombresListAmps.append(day)
    with open(fname, encoding='UTF-8') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',') # execution time in this step is limited by disk data transfer speed,
            # specially by the size of the bl and ac spike_list files
        ampdata[day] = dict()
        ampdata[day]['Electrode'] = []
        ampdata[day]['Amplitude'] = []
        for i, row in enumerate(csv_reader):
            if i > 0:
                try:
                    ampdata[day]['Electrode'].append(row[3]) # retrieve full electrode names
                    ampdata[day]['Amplitude'].append(float(row[4]))
                except IndexError:
                    break

        wells, coloring, treatment, concentration = None, None, None, None # well info is at the bottom of the table, below the last amplitude values
        for i, row in enumerate(csv_reader):
            try:
                if row[0] == 'Well':
                    wells = row[1:]
                if row[0] == 'Well Coloring':
                    well_coloring = row[1:]
                if row[0] == 'Treatment':
                    well_treatments = row[1:]
                if row[0] == 'Concentration':
                    well_concentrations = row[1:]
            except IndexError:
                break
                       
        for i, row in enumerate(csv_reader): # try again, necessary in files with very few amplitude values (headers are longer than values list)
            try:
                if row[0] == 'Well':
                    wells = row[1:]
                if row[0] == 'Well Coloring':
                    well_coloring = row[1:]
                if row[0] == 'Treatment':
                    well_treatments = row[1:]
                if row[0] == 'Concentration':
                    well_concentrations = row[1:]
            except IndexError:
                break
                
        coloring = [well_coloring[a] for a, w in enumerate(wells) for e in electrodes if e[:2] == w]
        treatment = [well_treatments[a] for a, w in enumerate(wells) for e in electrodes if e[:2] == w]
        concentration = [well_concentrations[a] for a, w in enumerate(wells) for e in electrodes if e[:2] == w]
        
        electrode_info[day] = dict(zip(electrodes, zip(coloring, treatment, concentration)))
print('\nCreated "ampdata" and "electrode_info" dictionaries')

In [ ]:
# Convert amplitude values from mV to μV and assign them to their electrode of origin
amps_per_electrode_in_uV = dict()
for day, amp_day_data in tqdm_notebook(ampdata.items(), desc = "Converting each day\'s amplitude values to μV and assigning them to each electrode on " + str(time.ctime(time.time()))):
    amps_per_electrode_in_uV[day] = defaultdict(list, {e:[] for e in electrodes})
    for el, amp in zip(amp_day_data['Electrode'], amp_day_data['Amplitude']):
        amps_per_electrode_in_uV[day][el].append(amp*1000)

In [ ]:
# Sort columns by treatment and concentration
if any("ac" in k.lower() for k in amps_per_electrode_in_uV.keys()): # if acute treatment is present in days list
    treatments = np.array([(e, conc+' '+t) if conc != '' else (e, t) for e, (color, t, conc) in electrode_info[nombresListAmps[-1]].items()], dtype=str).T # retrieve info from the last recording
    idx = np.argsort(treatments)[1]
else:
    print('This experiment does not have an acute treatment point, must be a development experiment or have faulty names')
    treatments = np.array([(e, t) for e, (color, t, conc) in electrode_info[list(electrode_info.items())[0][0]].items()], dtype=str).T
    idx = np.argsort(treatments)[1]

print('\nEach electrode\'s unsorted treatments are: ',  treatments[1])
print('\nFinal order of electrodes, sorted by treatment is: ', idx)
print('\nEach electrode\'s sorted treatments are: ', treatments[1][idx])

col_numbers = dict(zip(treatments[0][idx], range(len(idx))))
print('\nElectrode:index key:value couples, sorted by treatment: ', col_numbers)

In [ ]:
# Call averageAmps function and perform the averaging on the desired special time points (bl, pre, ac)
amps_per_electrode_in_uV_mergedbl, amps_per_electrode_in_uV_merged_per_hourbl = averageAmps(amps_per_electrode_in_uV, electrodes, 'bl')
amps_per_electrode_in_uV_mergedpre, amps_per_electrode_in_uV_merged_per_hourpre = averageAmps(amps_per_electrode_in_uV, electrodes, 'pretreatment')
amps_per_electrode_in_uV_mergedac, amps_per_electrode_in_uV_merged_per_hourac = averageAmps(amps_per_electrode_in_uV, electrodes, 'ac')

In [ ]:
# Remake name|date dictionary using the new averaged time points and
# Merge the averaged results in the adequate order within the remaining non-averaged data
defampsdict = dict()
averagedtimepointsdicts = amps_per_electrode_in_uV_merged_per_hourbl|amps_per_electrode_in_uV_merged_per_hourpre|amps_per_electrode_in_uV_merged_per_hourac

for k, v in averagedtimepointsdicts.items():
    defampsdict[k] = averagedtimepointsdicts[k]
for k in datesnottobemergeddict.keys():
    defampsdict[k] = amps_per_electrode_in_uV[k]

# If the newly created 'defampsdict' is not empty and is different from the original 'amps_per_electrode_in_uV'
# (so there was an actual remapping of the timepoints and data), rename it like the original 'amps_per_electrode_in_uV'

if defampsdict and defampsdict != amps_per_electrode_in_uV:
    amps_per_electrode_in_uV = defampsdict
if list(cleandatesdict.keys()) != nombresListAmps:
    nombresListAmps = list(cleandatesdict.keys())

In [ ]:
# Define excel workbook name and directory
excelNameAmps = os.path.join(output_dir, 'amps.xlsx')
counter = 0
# Create excel workbook
start = time.time()
with xlsxwriter.Workbook(excelNameAmps, options= {'nan_inf_to_errors': True}) as workbook:
    with tqdm_notebook(ncols=800, desc="Writing amplitude cells to a workbook on " + str(time.ctime(time.time()))) as pbar:
    # Write raw worksheets
        sh = workbook.add_worksheet('Mean_Amplitude_μV')
        sh.write(1, 0, 'Treatment')
        sh.write(0, 0, 'Electrode')
        styles = dict()
        for electrode, (color, treatment, concentration) in electrode_info[nombresListAmps[-1]].items():
            styles[treatment] = workbook.add_format({'bg_color': color}) ## pick treatments and colors from last time point
        styles[''] = workbook.add_format({'bg_color': '#00FF00'})## define not-treated condition and color
        for electrode, j in col_numbers.items():
            sh.write(0, j + 1, electrode)
            t,c = electrode_info[nombresListAmps[-1]][electrode][1:]
            sh.write(1, j + 1, c + ' ' + t if c != '' else t)
        for i, amp_day_data in enumerate(amps_per_electrode_in_uV.values()):
            sh.write(i + 2, 0, nombresListAmps[i])
            for electrode, amps in amp_day_data.items():
                pbar.update(0.5)      # execution time in this step is limited by disk data transfer speed,
                counter += 0.5        # specially the size of the 'bl', 'pretreatment' and 'ac' spike_list files
                if amps != []:
                    sh.write(i + 2, col_numbers[electrode] + 1, np.nanmean(amps),
                         styles[electrode_info[nombresListAmps[i]][electrode][1]])
                else:
                    sh.write(i + 2, col_numbers[electrode] + 1, 0,
                         styles[electrode_info[nombresListAmps[i]][electrode][1]])
    # Write normalized worksheets
        nsh = workbook.add_worksheet('nMean_Amplitude_μV')
        nsh.write(1, 0, 'Treatment')
        nsh.write(0, 0, 'Electrode')
        for electrode, j in col_numbers.items():
            nsh.write(0, j + 1, electrode)
            t,c = electrode_info[nombresListAmps[-1]][electrode][1:]
            nsh.write(1, j + 1, c + ' ' + t if c != '' else t)
        for i, amp_day_data in enumerate(amps_per_electrode_in_uV.values()):
            nsh.write(i + 2, 0, nombresListAmps[i])
            for electrode, amps in amp_day_data.items():
                pbar.update(0.5)
                counter += 0.5
                # if numerator AND denominator ARE NOT empty...
                if amps != [] and amps_per_electrode_in_uV[baseline][electrode] != []:
                    if "pretreatment" in baseline.lower() and 'bl' in list(cleandatesdict.keys())[i]:
                        nsh.write(i + 2, col_numbers[electrode] + 1, np.nanmean(amps),
                                styles[electrode_info[nombresListAmps[i]][electrode][1]])
                    else:
                        if np.nanmean(amps)/np.nanmean(amps_per_electrode_in_uV[baseline][electrode]) > 4:
                            nsh.write(i + 2, col_numbers[electrode] + 1, 0,
                                styles[electrode_info[nombresListAmps[i]][electrode][1]])
                        else:
                            nsh.write(i + 2, col_numbers[electrode] + 1, np.nanmean(amps)/np.nanmean(amps_per_electrode_in_uV[baseline][electrode]),
                                styles[electrode_info[nombresListAmps[i]][electrode][1]])
                        
                # else if numerador AND denominador ARE empty...
                else:   
                    nsh.write(i + 2, col_numbers[electrode] + 1, 0,
                         styles[electrode_info[nombresListAmps[i]][electrode][1]])
end = time.time()
print('Created "amps.xlsx" Workbook,',f"Runtime of the program was {end - start} seconds")
## No RuntimeWarnings means no division of zero or by zero has been performed and excel file will be clean from formula errors

In [ ]:
# Move amps.xlsx sheets to final Tables_elec_multi.xlsx

wb1 = xl.load_workbook(filename=excelNameAmps)
meanamps = wb1.worksheets[0]
if any("bl" in k.lower() for k in amps_per_electrode_in_uV.keys()) or any("pretreatment" in k.lower() for k in amps_per_electrode_in_uV.keys()):
    nmeanamps = wb1.worksheets[1]

merged = xl.load_workbook(filename=excelName)
copymeanamps = merged.create_sheet(meanamps.title, index=0)
if any("bl" in k.lower() for k in amps_per_electrode_in_uV.keys()) or any("pretreatment" in k.lower() for k in amps_per_electrode_in_uV.keys()):
    copyNmeanamps = merged.create_sheet(nmeanamps.title, index=1)

with tqdm_notebook(ncols=800, total = counter,
                   desc="Moving amplitude cells to final excel " + str(time.ctime(time.time()))) as pbar:
    for row in meanamps:
        for cell in row:
            new_cell = copymeanamps.cell(row=cell.row, column=cell.col_idx, value= cell.value)
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)
                pbar.update(0.5)
                
    if any("bl" in k.lower() for k in amps_per_electrode_in_uV.keys()) or any("pretreatment" in k.lower() for k in amps_per_electrode_in_uV.keys()):
        for row in nmeanamps:
            for cell in row:
                new_cell = copyNmeanamps.cell(row=cell.row, column=cell.col_idx, value= cell.value)
                if cell.has_style:
                    new_cell.font = copy(cell.font)
                    new_cell.border = copy(cell.border)
                    new_cell.fill = copy(cell.fill)
                    new_cell.number_format = copy(cell.number_format)
                    new_cell.protection = copy(cell.protection)
                    new_cell.alignment = copy(cell.alignment)
                    pbar.update(0.5)
                    
for sheet in merged:
    sheet.sheet_view.tabSelected = False
merged.active = 0
merged.save(excelName)
os.remove(excelNameAmps)
print('Moved amplitude worksheets to {} workbook and deleted {} workbook'.format(excelName, 'amps.xlsx'))

In [ ]:
input("\nPlease, check the Test.xlsx file in the output folder before continuing. Press Enter to continue")

In [ ]:
# Read excel file as pandas dataframe
df = pd.read_excel(excelName, index_col=0, header=1, sheet_name=None)

In [ ]:
# Match treatments with their color using the last time point's color-treatment key-value pair.
# Check the result is correct and use next code cell after manually correcting the excel workbook info if not
try:
    co = {}
    for color, treatm in zip(day_data['Well Coloring'], day_data['Treatment']):
        co[treatm] = color
    temp = {v : k for k, v in co.items()}
    res = {v : k for k, v in temp.items()}
    print(res,'\n\nNumber of treatments is: {}\n'.format(len(res)))
    
# Solution for when the treatment names are wrong in the .csv files but correct in the DataFrame after correction
# and the treatment-color pairs must be gotten from the dataframe (using 'Number of Spikes' sheet)
except KeyError:
    
    tem = list(df['Number_of_Spikes'].columns)
    col_headers = []
    # ran = [format(x, '0') for x in range(0,10)]
    for e in tem:
        if e.count('.') > 1:
            col_headers.append(e.split(' ', maxsplit=1)[0]+ ' ' + e.split(' ', maxsplit=1)[1].split('.')[0])
        elif '.' not in e:
            col_headers.append(e)
        elif '.' in e[:3] and '.' not in e[-1:-3]:
            col_headers.append(e)
        else:
            col_headers.append(e.split('.')[0])

    print(col_headers)
    wob = xl.load_workbook(excelName, data_only = True)
    sh = wob['Number_of_Spikes']
    hex_colors = []
    for column in range(2, sh.max_column+1):
        hex_colors.append(sh.cell(3,column).fill.start_color.index.replace('FF','#',1))
    co = {}
    for color, treatm in zip(hex_colors, col_headers):
        co[treatm] = color
    temp = {v : k for k, v in co.items()}
    res = {v : k for k, v in temp.items()}
    print(res)
    print ('HEX =', hex_colors) 
    #print('RGB =', tuple(int(hex_colors[i:i+2], 16) for i in (0, 2, 4))) # Color in RGB

# Create parameter data and SEM averages for each treatment directly from pandas dataframe content
'''These calculations are also performed within the descriptive plots' loop'''
parameters = dict()
sem = dict()
for metric_name, df_metric in tqdm_notebook(df.items(), desc = 'Creating "parameters" and "sem" data dictionaries on ' + str(time.ctime(time.time()))):
    parameters[metric_name] = {k: [] for k in res}
    sem[metric_name] = {k: [] for k in res}
    for day in df_metric.index:
        for trt in parameters[metric_name]:
            vals_over_trt = np.array([df_metric.loc[day][key] for key in df_metric.keys() if key.startswith(trt)])
            with np.errstate(invalid='ignore'):
                parameters[metric_name][trt].append(np.nanmean(vals_over_trt))
                sem[metric_name][trt].append(stats.sem(vals_over_trt, axis=0, ddof=1, nan_policy='omit'))

In [ ]:
# Call a function to create a composition with all the charts using add_gridspec and add_subplot
# statistics_chart(df)

In [ ]:
# Specify plot presets

titles_and_axes_on = FontProperties(family='Arial', style='normal', weight=250, stretch='normal', size=48)
legends_on = FontProperties(family='Arial', style='normal', weight=250, stretch='normal', size=42)

devel = []
overnight = []
after = []
post_on = []
for k in realistic_x_axis.keys():
    if 'bl' in k:
        break
    devel.append(k)

last_overnight = None
first_div = None
for i, (k, v) in enumerate(realistic_x_axis.items()):
    if 'bl' in k or 'ac' in k or 'on' in k or 'pretreatment' in k or 'Pretreatment' in k:
        overnight.append(k)
    else:
        last_overnight = list(realistic_x_axis.values())[i-1]
        first_div = v
        break

if first_div is not None and last_overnight is not None and abs(last_overnight - first_div) < 3:
    overnight.append(df['Number_of_Spikes'].index.values[len(overnight)]) # this line applies to the last timepoint of the on whose name is changed to 'div' something.
devel = np.array(devel)                                         # Sometimes a separate 11 min segment is recorded a few hours after the last 11 min of overnight recording
after = df['Number_of_Spikes'].index.values[len(devel):]
post_on = df['Number_of_Spikes'].index.values[len(devel) + len(overnight):]
print('Stablished line plot presets on ' + str(time.ctime(time.time())))

In [ ]:
# Plot post-treatment values with SEM whiskers and a broken (split) x-axis
workbook = xl.load_workbook(excelName)
prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]
r=0
with plt.ion():
    for name, mean_metr in tqdm_notebook(parameters.items(), total=((len(parameters.keys()))) , desc='Plotting and saving graphs on ' + str(time.ctime(time.time()))):
        if post_on.size == 0:
            fig1, axs1 = plt.subplots(1, 1, figsize=(40, 15), gridspec_kw=dict(hspace=0, wspace=0), sharex='none', sharey=True)
            axs1.patch.set_alpha(0.0)
            realistic_x = []
            for trt, row in mean_metr.items():
                if name[0] != 'n':
                    x = [df[name].index.get_loc(ind) for ind in after]
                elif name[0] == 'n' and any('pretreatment' in ind.lower() for ind in after):
                    x = [df[name].index.get_loc(ind) for ind in after if 'bl' not in ind.lower()]
                realistic_x = [list(realistic_x_axis.values())[i] for i in x]
                axs1.errorbar(realistic_x, np.nan_to_num(np.array(row)[x], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x], copy=False),
                              label=trt, elinewidth=1.4, linewidth=2.25, color='black', marker='o', markersize='24', zorder=3,
                              barsabove=False, capthick=2, markeredgecolor='black',
                              markerfacecolor=res[trt], fmt = ' ', capsize=18, ecolor=res[trt])
                axs1.errorbar(realistic_x, np.nan_to_num(np.array(row)[x], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x], copy=False), label=trt,
                              elinewidth=2.2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=False, capthick=2.8, markeredgecolor='black',
                              markerfacecolor=res[trt], capsize=18.5, ecolor='black')
                handles, labels = axs1.get_legend_handles_labels()
                handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
                by_label = dict(zip(labels, handles))
                axs1.legend(by_label.values(), by_label.keys(), ncol=len(res), labelspacing=1, columnspacing=2,
                            bbox_to_anchor=(0.20, 1.05), loc="lower left", framealpha=0, prop=legends_on)
        else:
            custom_fig_size=(40, 15)
            fig1, (axs1,axs2) = plt.subplots(1, 2, figsize=custom_fig_size, gridspec_kw=dict({'width_ratios': [len(overnight), len(post_on)]}, hspace=0, wspace=0.03),
                                             sharex='none', sharey=True, facecolor='w')
            axs1.patch.set_alpha(0.0)
            axs2.patch.set_alpha(0.0)
            realistic_x = []
            for trt, row in mean_metr.items():
                if name[0] != 'n':
                    x = [df[name].index.get_loc(ind) for ind in after]
                elif name[0] == 'n' and any('pretreatment' in ind.lower() for ind in after):
                    x = [df[name].index.get_loc(ind) for ind in after if 'bl' not in ind.lower()]
                x2 = [df[name].index.get_loc(ind) for ind in post_on]
                realistic_x = [list(realistic_x_axis.values())[i] for i in x]
                realistic_x2 = [list(realistic_x_axis.values())[i] for i in x2]
                axs1.errorbar(realistic_x, np.nan_to_num(np.array(row)[x], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x], copy=False),
                              label=trt, elinewidth=1.4, linewidth=2.25, color='black', marker='o', markersize='24', zorder=3,
                              barsabove=False, markeredgecolor='black', markerfacecolor=res[trt], fmt = ' ',
                              capsize=18.4, capthick=1.2, ecolor=res[trt])
                axs1.errorbar(realistic_x, np.nan_to_num(np.array(row)[x], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x], copy=False),
                              label=trt, elinewidth=2.2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=False, markeredgecolor='black', markerfacecolor=res[trt],
                              capsize=18.8, capthick=2.6, ecolor='black')
                axs2.errorbar(realistic_x2, np.nan_to_num(np.array(row)[x2], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x2], copy=False),
                              label=trt, elinewidth=1.4, linewidth=2.25, color='black', marker='o', markersize='24', zorder=3,
                              barsabove=False, markeredgecolor='black', markerfacecolor=res[trt], fmt = ' ',
                              capsize=18.4, capthick=1.2, ecolor=res[trt])
                axs2.errorbar(realistic_x2, np.nan_to_num(np.array(row)[x2], copy=False),
                              yerr = np.nan_to_num(np.array(sem[name][trt])[x2], copy=False),
                              label=trt, elinewidth=2.2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=False, markeredgecolor='black', markerfacecolor=res[trt],
                              capsize=18.8, capthick=2.6, ecolor='black')
            
            axs2.set_xlim(realistic_x_axis[post_on[0]]-6, realistic_x_axis[post_on[-1]] + 1)
            plt.setp(axs2, xticks=np.arange(round(realistic_x_axis[post_on[0]]), round(realistic_x_axis[post_on[-1]] + 50), 50))
            axs2.tick_params(axis='x', length=25, width=3.5, pad=12)
            plt.setp(axs2.xaxis.get_majorticklabels(), rotation=0, fontproperties=titles_and_axes_on)
            axs2.spines['left'].set_visible(False)
            axs2.spines['bottom'].set_linewidth(3.5)
            axs2.spines['top'].set_visible(False)
            axs2.spines['right'].set_visible(False)
            axs2.yaxis.set_visible(False)
            axs2.grid(False)
            d = .025 # how big to make the diagonal lines in axes coordinates
            kwargs = dict(transform=axs1.transAxes, color='k', clip_on=False, linewidth=3.5)
            axs1.plot((1,1), (-d,+d), **kwargs)
            kwargs.update(transform=axs2.transAxes)
            axs2.plot((0,0), (-d,+d), **kwargs)
            handles, labels = axs1.get_legend_handles_labels()
            handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
            by_label = dict(zip(labels, handles))
            axs1.legend(by_label.values(), by_label.keys(), ncol=len(res), labelspacing=1, columnspacing=2,
                        bbox_to_anchor=(0.6, 1.05), loc="lower center", framealpha=0, prop=legends_on)

        # Settings common to both cases
        axs1.set_xlim(realistic_x_axis[overnight[0]]-0.5, realistic_x_axis[overnight[-1]]+0.5)
        plt.setp(axs1, xticks=np.arange(0, round(realistic_x_axis[overnight[-1]]+1.51), 1))
        plt.setp(axs1.yaxis.get_majorticklabels(), fontproperties=titles_and_axes_on)
        plt.setp(axs1.xaxis.get_majorticklabels(), rotation=0, fontproperties=titles_and_axes_on)
        axs1.tick_params(axis='both', labelright='off', length=25, width=3.5, pad=12)
        axs1.spines['left'].set_linewidth(3.5)
        axs1.spines['bottom'].set_linewidth(3.5)
        axs1.spines['top'].set_visible(False)
        axs1.spines['right'].set_visible(False)
        axs1.yaxis.tick_left()
        axs1.grid(False)
        oldname = name
        if name[0] == 'n':
            name = re.sub('^n', 'Normalized ', name) # use '^n' to replace the first lowercase n by 'Normalized',there is no length limit unlike for excel sheet names
            name = re.sub(r' \([^()]*\)', '', name) # use ' (*' as a marker to delete units (all units in names are inside parentheses)
        fig1.text(0.5, -0.02, 'Time (hours)', ha='center', fontproperties=titles_and_axes_on)
        axs1.set_ylabel(name, labelpad=35, fontproperties=titles_and_axes_on)
        # Set bottom ylimit to 0 if ylim[0] > 0
        ymin, ymax = axs1.get_ylim()
        if ymin > 0:
            axs1.set_ylim(ymin=0, ymax=ymax*1.1)
        else:
            pass
        axs1.annotate('', xy=(realistic_x_axis[[k for k in realistic_x_axis.keys() if 'ac' in k][0]], axs1.get_ylim()[0]),
                      xytext=(realistic_x_axis[[k for k in realistic_x_axis.keys() if 'ac' in k][0]], axs1.get_ylim()[1]), rotation=0,
                      ha='center', va='center', annotation_clip=False, zorder=1,
                      arrowprops={"color" : "black", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

        axs1.text(realistic_x_axis[[k for k in realistic_x_axis.keys() if 'ac' in k][0]]+0.15, axs1.get_ylim()[1]*0.89,
                  'DIV'+str(day_of_treatment), rotation=90, fontname='Arial', color='black', fontsize=40, ha='left', va='baseline') # if text separates more than 0.2 from the line is because of autoshrink
        if post_on.size > 0:
            axs2.annotate('', xy=(realistic_x_axis[post_on[0]], axs2.get_ylim()[0]),
                          xytext=(realistic_x_axis[post_on[0]], axs2.get_ylim()[1]), rotation=0,
                          fontsize=37, fontname='Arial', color='black', ha='center', va='center', annotation_clip=False, zorder=1,
                          arrowprops={"color" : "black", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

            axs2.text(realistic_x_axis[post_on[0]]+0.3, axs1.get_ylim()[1]*0.89,
                      post_on[0].upper(), rotation=90, fontname='Arial', color='black', fontsize=40, ha='left', va='baseline')
            axs2.annotate('', xy=(realistic_x_axis[post_on[-1]], axs2.get_ylim()[0]),
                          xytext=(realistic_x_axis[post_on[-1]], axs2.get_ylim()[1]), rotation=0,
                          fontsize=37, fontname='Arial', color='black', ha='center', va='center', annotation_clip=False, zorder=1,
                          arrowprops={"color" : "black", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

            axs2.text(realistic_x_axis[post_on[-1]]+0.3, axs1.get_ylim()[1]*0.89,
                      post_on[-1].upper(), rotation=90, fontname='Arial', color='black', fontsize=40, ha='left', va='baseline')

        # Save the figures as .png to folder within the loop
        if os.path.isdir(os.path.join(output_dir,'Graphs per electrode')) == False:
            os.mkdir(os.path.join(output_dir,'Graphs per electrode'))
        fig1.savefig(os.path.join(output_dir,'Graphs per electrode\\') + name, edgecolor='none', transparent=False)
        
        # Insert figures on the fly on their corresponding worksheet
        sheet = workbook[oldname]
        img = xl.drawing.image.Image(os.path.join(output_dir,'Graphs per electrode\\') + name + '.png')
        img.width = 23*custom_fig_size[0]
        img.height = 30*custom_fig_size[1]
        sheet.add_image(img, '{}{}'.format('A', len(cleandatesdict.keys())+4))
        
        # Save figures to a .pptx within the loop
        left=Inches(2.9)
        top=Inches(0.05)
        width=Inches(6.75)
        img_path = os.path.join(output_dir,'Graphs per electrode\\') + name + '.png'
        if r == 0:
            slide = prs.slides.add_slide(blank_slide_layout)
            pic = slide.shapes.add_picture(img_path, left, top, width)
            txBox = slide.shapes.add_textbox(Inches(0.1), Inches(0.1), Inches(3), Inches(0.5))
            tf = txBox.text_frame
            tf.text = name
            tf.paragraphs[0].font.bold = True
            tf.paragraphs[0].font.size = Pt(20)
            r=1
        elif r==1:
            top=Inches(3.75)
            pic = slide.shapes.add_picture(img_path, left, top, width)
            r=0
        
#         plt.show(close=None, block=None) # Inline plotting consumes many resources and hugely increases running time
#         prs.save('Presentation.pptx') # Save after each iteration so I can see the live evolution of the file

plt.close('all') # Close all figures and do not display any inline after running the loop (inline plotting is time-consuming)
prs.save(os.path.join(output_dir,'Graphs per electrode.pptx')) # Save pptx after the loop is done to save time
workbook.save(excelName)

# Move all .csv files to "analysis files" folder
# if os.path.isdir('Unsorted analysis files') == False:
#     os.mkdir('Unsorted analysis files')
# for fl in input_files:
#     if '.csv' in fl or '.txt' in fl:
#         shutil.move(fl, os.path.join('Unsorted analysis files', fl))